In [97]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing import image

Preprocessing

In [23]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

In [24]:
mnist_train, mnist_test  = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)

BUFFER_SIZE = 10000
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

Outline the Model

In [78]:
input_size = 784
output_size = 10
hidden_layer_size = 300

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(output_size, activation='softmax'),
                            ])

Optimizer and Loss Function

In [79]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Training

In [92]:
NUM_EPOCHS = 10

model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2)


Epoch 1/10
540/540 - 7s - 13ms/step - accuracy: 0.9968 - loss: 0.0124 - val_accuracy: 0.9963 - val_loss: 0.0104
Epoch 2/10
540/540 - 7s - 13ms/step - accuracy: 0.9979 - loss: 0.0085 - val_accuracy: 0.9978 - val_loss: 0.0090
Epoch 3/10
540/540 - 6s - 11ms/step - accuracy: 0.9976 - loss: 0.0097 - val_accuracy: 0.9917 - val_loss: 0.0540
Epoch 4/10
540/540 - 6s - 11ms/step - accuracy: 0.9970 - loss: 0.0130 - val_accuracy: 0.9965 - val_loss: 0.0143
Epoch 5/10
540/540 - 6s - 11ms/step - accuracy: 0.9974 - loss: 0.0120 - val_accuracy: 0.9982 - val_loss: 0.0078
Epoch 6/10
540/540 - 6s - 12ms/step - accuracy: 0.9982 - loss: 0.0088 - val_accuracy: 0.9968 - val_loss: 0.0082
Epoch 7/10
540/540 - 6s - 12ms/step - accuracy: 0.9974 - loss: 0.0106 - val_accuracy: 0.9920 - val_loss: 0.0474
Epoch 8/10
540/540 - 6s - 12ms/step - accuracy: 0.9972 - loss: 0.0119 - val_accuracy: 0.9967 - val_loss: 0.0096
Epoch 9/10
540/540 - 6s - 11ms/step - accuracy: 0.9982 - loss: 0.0069 - val_accuracy: 0.9985 - val_loss:

Testing

In [93]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 287ms/step - accuracy: 0.9831 - loss: 0.1520


In [94]:
# We can apply some nice formatting if we want to
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.15. Test accuracy: 98.31%


Test with your own Image

In [100]:
def load_and_preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(28, 28), color_mode='grayscale')
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = img / 255.0
    return img


def predict_image(image_path):
    img = load_and_preprocess_image(image_path)
    predictions = model.predict(img)
    predicted_digit = np.argmax(predictions)
    return predicted_digit

image_path = 'Two.png'
predicted_digit = predict_image(image_path)
print(f'The predicted digit is: {predicted_digit}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
The predicted digit is: 5
